In [0]:
from google.colab import drive
drive.mount('/content/drive')

#Requirements

In [0]:
%%writefile setup.sh

export CUD_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [0]:
#For apex
# !sh setup.sh

In [0]:
!pip install transformers --quiet

     |████████████████████████████████| 573kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 8.6MB/s 
     |████████████████████████████████| 890kB 16.2MB/s 
     |████████████████████████████████| 3.7MB 25.6MB/s 


#Imports

In [0]:
import warnings
warnings.simplefilter('ignore')

In [0]:
import os
import re
import gc
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

In [0]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [0]:
import transformers
from transformers import (AutoTokenizer, AutoModel, AdamW, AutoConfig, get_linear_schedule_with_warmup)

#Envs

In [0]:
seed = 2020 # for reproductibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available(): 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [0]:
os.makedirs('MODELS/', exist_ok=True)

#Utilities

In [0]:
def process_prediction(preds):
  r'''
    This function helps us go from a classifiaction
    problem to a regression one.
    The regression values range are in [-1, 1].
  '''

  final_preds = []
  for pred in preds:
    argmax = np.argmax(pred, axis=0)
    if argmax == 0: final_preds.append( -1*pred[0] )
    elif argmax == 1: final_preds.append( 0 )
    else: final_preds.append( pred[2] )
    
  return final_preds


def rmse(true, pred):
  return np.sqrt(mean_squared_error(true, pred))

In [0]:
def convert_examples(tweets, tokenizer, max_length): 
  bep = tokenizer.batch_encode_plus(tweets, add_special_tokens=True, max_length=max_length, 
                                    pad_to_max_length=True, return_token_type_ids=True,
                                    return_attention_masks=True)
  
  return bep['input_ids'], bep['token_type_ids'], bep['attention_mask']

In [0]:
class EarlyStopping:
  def __init__(self, patience=3, mode='min'):
    self.step = 0
    self.patience = patience
    self.mode = mode
    self.stop = False
    self.loss = np.inf

  def update(self, loss):
    if self.loss < loss:
      self.step += 1
    else: 
      self.step = 0
      self.loss = loss
    
    if self.step == self.patience: self.stop = True 

In [0]:
class VaccineOrNotModel(nn.Module):
    def __init__(self, model_name):
        super(VaccineOrNotModel, self).__init__()

        config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(config.hidden_size, 3)
        nn.init.xavier_normal_(self.classifier.weight)
    
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        _, out = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        out = self.dropout(out)
        logits = self.classifier(out)
        logits = F.sigmoid(logits)

        return logits

In [0]:
class VaccineOrNotDataset(Dataset):
  def __init__(self, input_ids, att_masks, token_ids, labels=None, task='train', classes=3):
    self.input_ids = input_ids
    self.att_masks = att_masks
    self.token_ids = token_ids
    self.labels = labels
    self.c = classes
    self.task = task

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, id):
    out = {
        'input_ids': torch.tensor(self.input_ids[id]),
        'token_type_ids': torch.tensor(self.token_ids[id]),
        'attention_mask': torch.tensor(self.att_masks[id]),
    }

    if self.task=='train': 
      out.update(
        {
          'label': torch.tensor(to_categorical(self.labels[id]+1, self.c), dtype=torch.float), #The former labels are in [-1, 1], so we add +1 to be in [0, 2]
        }
      )

    return out

In [0]:
def training_fn(model, opt, scheduler, criterion, train_ds, bs, device='cuda', n_labels=3):
  train_dl = DataLoader(train_ds, bs)

  avg_rmse = 0
  avg_loss = 0

  pbar = tqdm(train_dl, desc='Training ...')
  model.train()

  for i, data in enumerate(pbar):
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = data['label'].to(device)

    opt.zero_grad()

    logits = model(input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids)
        
    loss = criterion(logits, labels)
    custom_loss = rmse( labels.cpu().detach().numpy().argmax(1)-1, process_prediction(logits.cpu().detach().numpy()) )

    avg_loss += loss.item()
    avg_rmse += custom_loss

    loss.backward()

    opt.step()
    scheduler.step()

    pbar.set_postfix(loss=avg_loss/(i+1), rmse=avg_rmse/(i+1))
    pbar.update()

  return avg_rmse/len(train_dl)

In [0]:
def eval_fn(model, criterion, valid_ds, eval_bs, device='cuda', n_labels=3):
  valid_dl = DataLoader(valid_ds, eval_bs)

  avg_rmse = 0
  avg_loss = 0

  pbar = tqdm(valid_dl, desc='Evaluation ...')
  model.eval()

  for i, data in enumerate(pbar):
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = data['label'].to(device)

    logits = model(input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids
        )
    

    loss = criterion(logits, labels)
    
    custom_loss = rmse( labels.cpu().detach().numpy().argmax(1)-1, process_prediction(logits.cpu().detach().numpy()) )

    avg_loss += loss.item()
    avg_rmse += custom_loss

    pbar.set_postfix(loss=avg_loss/(i+1), rmse=avg_rmse/(i+1))
    pbar.update()

  return avg_rmse/len(valid_dl)

In [0]:
def predict(input_ids, mask, token_ids, MODELS, bs=8, n_labels=3):
    test_ds = VaccineOrNotDataset(input_ids, mask, token_ids, task='test')
    testloader = DataLoader(test_ds, bs, shuffle=False)

    predictions_proba = []

    out = None

    for data in tqdm(testloader):
        x = data['input_ids'].to(device)

        for i in range(n_folds):
            if i == 0: out = MODELS[i](input_ids=x)
            else: out += MODELS[i](input_ids=x)

        out /= n_folds
        out_probas = out.cpu().detach().numpy()

        predictions_proba += out_probas.tolist()

    return predictions_proba


In [0]:
def predict_by_fold(model, input_ids, mask, token_ids, bs=8, n_labels=3):
    test_ds = VaccineOrNotDataset(input_ids, mask, token_ids, task='test')
    testloader = DataLoader(test_ds, bs, shuffle=False)

    predictions_proba = []

    out = None

    for data in tqdm(testloader):
        x = data['input_ids'].to(device)
        
        out = model(input_ids=x)

        out /= n_folds
        out_probas = out.cpu().detach().numpy()

        predictions_proba += out_probas.tolist()

    return predictions_proba


In [0]:
def run_fold(fold, epochs, splits, bs, eval_bs, lr, model_name, path='MODELS/'):
  tr, vr = splits
  train_ds = VaccineOrNotDataset(np.take(tr_ids, tr, axis=0), np.take(tr_masks,tr, axis=0),
                                np.take(tr_tokens,tr, axis=0), np.take(train_labels,tr, axis=0)) 
  valid_ds = VaccineOrNotDataset(np.take(tr_ids, vr, axis=0), np.take(tr_masks,vr, axis=0),
                                np.take(tr_tokens,vr, axis=0), np.take(train_labels,vr, axis=0)) 

  train_dl = DataLoader(train_ds, bs)
  valid_dl = DataLoader(valid_ds, eval_bs)

  device='cuda'

  model = VaccineOrNotModel(model_name)
  model.to(device)

  num_train_steps = int(len(train_ds) / bs*epochs)

  param_optimizer = list(model.named_parameters())
  no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
  optimizer_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
  ]

  opt = AdamW(optimizer_parameters, lr=lr)
  scheduler = get_linear_schedule_with_warmup(
            opt, num_warmup_steps=0, num_training_steps=num_train_steps
        )

  criterion = nn.BCELoss()
  es = EarlyStopping(patience=3)

  best_rmse = np.inf

  for epoch in tqdm(range(epochs)):
    training_fn(model, opt, scheduler, criterion, train_ds, bs, device)
    score = eval_fn(model, criterion, valid_ds, eval_bs, device)
    es.update(score)

    if score < best_rmse:
      best_rmse = score
      torch.save(model.state_dict(), f'{path}model_state_dict_{fold}.bin')

    if es.stop:
      print("Early Stopping triggered")
      return best_rmse
      
  return best_rmse

#Reading the Data

In [0]:
path = 'drive/My Drive/Zindi/#ZindiWeekendz/'

In [0]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')
sample = pd.read_csv(path+'SampleSubmission.csv')

#Cleaning and Tokenization

In [0]:
model_name = 'roberta-large'

In [0]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [0]:
train['safe_text'] = train['safe_text'].apply(str)
test['safe_text'] = test['safe_text'].apply(str)

In [0]:
train['safe_text'] = train['safe_text'].apply(str.lower)
test['safe_text'] = test['safe_text'].apply(str.lower)

In [0]:
train['safe_text'] = train['safe_text'].apply(lambda x: x.replace('&amp;', ' '))
test['safe_text'] = test['safe_text'].apply(lambda x: x.replace('&amp;', ' '))

In [0]:
train['safe_text'] = train['safe_text'].apply(lambda x: x.replace('<user>', ' '))
test['safe_text'] = test['safe_text'].apply(lambda x: x.replace('<user>', ' '))

train['safe_text'] = train['safe_text'].apply(lambda x: x.replace('<url>', ' '))
test['safe_text'] = test['safe_text'].apply(lambda x: x.replace('<url>', ' '))

In [0]:
train['safe_text'] = train['safe_text'].apply(lambda x: x.replace('#', ' '))
test['safe_text'] = test['safe_text'].apply(lambda x: x.replace('#', ' '))

In [0]:
train['safe_text'] = train['safe_text'].apply(lambda x: x.strip('.').strip())
test['safe_text'] = test['safe_text'].apply(lambda x: x.strip('.').strip())

In [0]:
train.drop(index=[4798, 4799], inplace=True)
train.reset_index(drop=True, inplace=True)

In [0]:
train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,me the big homie meanboy3000 meanboy mb m...,0.0,1.0
1,E3303EME,i'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"whatcausesautism vaccines, do not vaccinate yo...",-1.0,1.0
3,1DR6ROZ4,i mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,thanks to catch me performing at la nuit nyc...,0.0,1.0


In [0]:
train_text = train['safe_text'].values
train_labels = train['label'].values

test_text = test['safe_text'].values

In [0]:
tr_ids, tr_tokens, tr_masks = convert_examples(train_text, tokenizer, max_length=34)

In [0]:
te_ids, te_tokens, te_masks = convert_examples(test_text, tokenizer, max_length=34)

#Training

In [0]:
#These Hyperparameters could be tune
epochs = 10
n_folds = 10
bs = 32
eval_bs = 4
lr = 3e-5
device = 'cuda'

In [0]:
skf = StratifiedKFold(n_folds, shuffle=True, random_state=seed)
all_folds = list(skf.split(tr_ids, train_labels))

In [0]:
cv_score = 0
for fold in range(n_folds):
  splits = all_folds[fold]

  s = run_fold(fold, epochs, splits, bs, eval_bs, lr, model_name=model_name)
  cv_score += s/n_folds
print("Avg rmse : ", cv_score)

#Prediction

In [0]:
#Just to cool down the memory :)
del train,tokenizer,tr_ids,tr_masks,tr_tokens
gc.collect()

In [0]:
preds = []

for fold in range(n_folds):
  model = VaccineOrNotModel(model_name)
  model.to(device)
  model.load_state_dict(torch.load(f'MODELS/model_state_dict_{fold}.bin'))
  model.eval()

  preds.append( predict_on_fold(model, te_ids, te_masks, te_tokens, bs=8) )

  del model
  gc.collect()

In [0]:
predictions_proba = np.sum(preds, axis=0)

In [0]:
predictions = process_prediction(predictions_proba) #Final step

#Submission

In [0]:
subs = test[['tweet_id']]
subs['target'] = predictions
subs.columns = ['ID', 'target']
subs.head()

,ID,target
0,00BHHHP1,-0.518568
1,00UNMD0E,0.000000
2,01AXPTJF,0.000000
3,01HOEQJW,0.921080
4,01JUKMAO,0.000000


In [0]:
subs.describe()

,target
count,5177.000000
mean,0.305531
std,0.480103
min,-0.886795
25%,0.000000
50%,0.000000
75%,0.841104
max,0.935912


In [0]:
subs.to_csv(f'arch_{model_name}_folds_{n_folds}_epochs_{epochs}_bs_{bs}_lr_{lr}_cv_{cv_score}.csv', index=False)